In [ ]:
!pip install crewai
!pip install 'crewai[tools]'

In [ ]:
!pip install langchain-community

In [ ]:
from crewai_tools import GithubSearchTool

In [ ]:
import os
from getpass import getpass


os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI_API_KEY: ")

Enter OPENAI_API_KEY: ··········


In [ ]:
from crewai_tools import GithubSearchTool

# Initialize the tool for semantic searches within a specific GitHub repository

github_repo='https://github.com/aiplanethub/beyondllm.git'
search_query = 'Review Python code in Github Repo and provide feedback on issues and corrections that need to be made. Be sure to name each file and provide a brief description of the issues found.'


gh_token = getpass("Enter GitHub Access Token: `github_pat_...`: ")

github_tool = GithubSearchTool(
    github_repo=github_repo,
    gh_token=gh_token,
    search_query=search_query,
    content_types=['code',],
    config=dict(
        llm=dict(
            provider="openai",
            config=dict(
                model="gpt-4-turbo"
            ),
        ),
        embedder=dict(
            provider="openai",
            config=dict(
                model="text-embedding-3-small",
            ),
        ),
    ),
)

Enter GitHub Access Token: `github_pat_...`: ··········


Searching data from github:   0%|          | 0/1 [00:00<?, ?it/s]
Loading code files from github: 0it [00:00, ?it/s]
Searching data from github: 100%|██████████| 1/1 [00:00<00:00,  1.27it/s]


In [ ]:
from crewai import Agent
from langchain_community.chat_models import ChatOpenAI
researcher = Agent(

  role='Senior Researcher',

  goal='Get all the code from the given repository given',

  verbose=True,

  memory=True,

  backstory=(
    "Has the capability to fetch data from a given repository"
  ),

  tools=[github_tool],

  allow_delegation=True,
  llm=ChatOpenAI(model_name="gpt-4-turbo", temperature=0.8)

)

In [ ]:
writer = Agent(
  role='Writer',

  goal='Writes detailed documentation for the repository',

  verbose=True,

  memory=True,

  backstory=("Expert in writing detailed documentation"

  ),

  allow_delegation=False,
  llm=ChatOpenAI(model_name="gpt-4-turbo", temperature=0.8)

)

In [ ]:
from crewai import Task



# Research task

research_task = Task(

  description=(
    "To fetch all the data including coding files as well as the readme and get them ready for the writing of the documentation"

  ),

  expected_output='All the required files for writing the documentation',

  tools=[github_tool],

  agent=researcher,

)



# Writing task with language model configuration

write_task = Task(

  description=(
"Compose a comprehensive documentation for the repository which has code snippets and user guide on how to use it"

  ),

  expected_output='A comprehensive documentation',


  agent=writer,

  async_execution=False,

  output_file='documentation.txt'
  )# Example of output customization


In [ ]:
from crewai import Crew, Process

# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True
)

In [ ]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff()
print(result)



> Entering new CrewAgentExecutor chain...
Thought: Before I proceed to fetch all the coding files and the readme, I need to understand the structure and contents of the GitHub repository to ensure that I retrieve all necessary files for documentation. I'll first search the repository to get an overview of its structure, file types, and any specific coding files and readme files available.

Action: Search a github repo's content
Action Input: {"search_query": "file structure"} 

Relevant Content:
the muck" optional = false python-versions = "*" files = [ {file = "dirtyjson-1.0.8-py3-none-any.whl", hash = "sha256:125e27248435a58acace26d5c2c4c11a1c0de0a9c5124c5a94ba78e517d74f53"}, {file = "dirtyjson-1.0.8.tar.gz", hash = "sha256:90ca4a18f3ff30ce849d100dcf4a003953c79d3a2348ef056f1d9c22231a25fd"}, ] [package] name = "distro" version = "1.9.0" description = "Distro - an OS platform information API" optional = false python-versions = ">=3.6" files = [ {file = "distro-1.9.0-py3-none-any.whl"